In [1]:
## http://web.stanford.edu/~lwhsiao/api/
%load_ext autoreload

%autoreload 2
%matplotlib inline

import os
import sys

PARALLEL = 4 # assuming a quad-core machine
ATTRIBUTE = "entity_phone"
#os.environ['SNORKELDB']
os.environ['SNORKELDBNAME'] = "price_extractor_texas"
print os.environ['SNORKELDBNAME']
os.environ['SNORKELDB'] = 'postgresql://localhost:5432/' + os.environ['SNORKELDBNAME']


sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/fonduer/memex/')

price_extractor_texas


###  Defining a Candidate Schema

In [2]:
from snorkel.contrib.fonduer import SnorkelSession

session = SnorkelSession()

In [3]:
import os
from snorkel.contrib.fonduer.models import candidate_subclass

Published_Hourly_Price = candidate_subclass('published_hourly_price', ["price"])


### Parsing and Transforming the Input Documents into Unified Data Models

In [4]:
from snorkel.contrib.fonduer import HTMLPreprocessor, OmniParser

docs_path = os.environ['SNORKELHOME'] + '/tutorials/fonduer/memex/data/texas_profiles_data/'

doc_preprocessor = HTMLPreprocessor(docs_path)

### Configuring an `OmniParser`


In [5]:
corpus_parser = OmniParser(structural=True, lingual=True)
%time corpus_parser.apply(doc_preprocessor, parallelism=PARALLEL)

Clearing existing...
Running UDF...
CPU times: user 10.3 s, sys: 327 ms, total: 10.6 s
Wall time: 3min 58s


In [6]:
from snorkel.contrib.fonduer.models import Document, Phrase,Table

print "Documents:", session.query(Document).count()
print "Phrases:", session.query(Phrase).count()
print "Table", session.query(Table).count()


Documents: 342
Phrases: 71150
Table 832


## Dividing the Corpus into Test and Train

In [7]:
docs = session.query(Document).order_by(Document.name).all()
ld   = len(docs)

train_docs = set()
dev_docs   = set()
test_docs  = set()
splits = (0.8, 0.9)
data = [(doc.name, doc) for doc in docs]
data.sort(key=lambda x: x[0])
for i, (doc_name, doc) in enumerate(data):
    if i < splits[0] * ld:
        train_docs.add(doc)
    elif i < splits[1] * ld:
        dev_docs.add(doc)
    else:
        test_docs.add(doc)
from pprint import pprint
#pprint([x.name for x in train_docs])
print "train:",len(train_docs)
print "dev:" ,len(dev_docs)
print "test:",len(test_docs)

train: 274
dev: 34
test: 34


In [8]:
from snorkel.matchers import *
import re

number_matcher = NumberMatcher(longest_match_only=True) 
#price_regex = re.compile(r'[0-9]+[\.\,]?[0-9]*[:blank:]*\$')
#price_matcher_1= RegexMatchSpan(rgx =u'[0-9]+[\.\,]?[0-9]*[:blank:]*\$', longest_match_only = True)
#price_matcher_2= RegexMatchSpan(rgx =u'$\[0-9]+[\.\,]?[0-9]*[:blank:]', longest_match_only = True)

####Define a relation's ContextSpaces

from snorkel.contrib.fonduer.fonduer.candidates import OmniNgrams
price_matcher= RegexMatchSpan(rgx =u'(\d+).(\d+)', longest_match_only = True)

##part_matcher = Union(price_matcher_1, price_matcher_2)
price_ngrams = OmniNgrams(n_max=6, split_tokens=[])


### Defining candidate Throttlers

In [40]:
from snorkel.contrib.fonduer.lf_helpers import *
from snorkel.contrib.fonduer.fonduer.lf_helpers import *
def right_ngrams_price_filter(cand):
    lst = ['/hour','hr','hour']

    cand_right_negrams= list(get_right_ngrams(cand, window =3))
   
    if "half" not in cand_right_negrams:
        for token in cand_right_negrams:
            if token in lst:
                return cand
    else:
        return False
            
        
            
candidate_filter=right_ngrams_price_filter

In [41]:
from snorkel.contrib.fonduer.candidates import CandidateExtractor

candidate_extractor = CandidateExtractor(Published_Hourly_Price,
                                         [price_ngrams], [price_matcher],
                                         candidate_filter=candidate_filter)


#                         candidate_filter=candidate_filter

%time candidate_extractor.apply(train_docs, split=0, parallelism=PARALLEL)

Clearing existing...
Running UDF...
CPU times: user 89.7 ms, sys: 62.2 ms, total: 152 ms
Wall time: 8.33 s


In [42]:
train_cands = session.query(Published_Hourly_Price).filter(Published_Hourly_Price.split == 0).all()
print "Number of candidates:", len(train_cands)

Number of candidates: 186


In [43]:
%%time
for i, docs in enumerate([dev_docs, test_docs]):
    candidate_extractor.apply(docs, split=i+1)
    print "Number of candidates:", session.query(Published_Hourly_Price).filter(Published_Hourly_Price.split == i+1).count()
    
    

Clearing existing...
Running UDF...
[========================================] 100%
Number of candidates: 27
Clearing existing...
Running UDF...
[========================================] 100%
Number of candidates: 22
CPU times: user 3.69 s, sys: 175 ms, total: 3.86 s
Wall time: 4.48 s


In [47]:
train_cands = session.query(Published_Hourly_Price).filter(Published_Hourly_Price.split==0).all()
#session.query(Location_Extraction).filter(Location_Extraction.split == 0).all()
print len(train_cands)
cands9= train_cands[9]
print "text for the 9th candidate:\n", cands9.get_parent()
print "9th candidate\n:",cands9
cands10= train_cands[10]
print "text for the 10th candidate:\n", cands10.get_parent()
print "10th candidate\n:",cands10
print "#########################"
cands12= train_cands[12]
print "text for the 12th candidate:\n", cands12.get_parent()
print "12th candidate\n:",cands12
print "########################################"
# ance_16 = get_ancestor_tag_names(cands10)
# print "ancestor of 16th candidate\n:", ance_16 
cands100= train_cands[100]
print "text for the 100th candidate:\n", cands100.get_parent()
print "100th candidate\n:",cands100
print "###################################################"

all_cands = session.query(Published_Hourly_Price).all()
len(all_cands)

186
text for the 9th candidate:
Phrase (Doc: 57f9a1f2-9109-4038-bdf5-b6e49a9632fe, Index: 52, Text: $ 150.00   /half hour,       $ 250.00   /hour     ,   $ 500.00   /2 hours    ,   $ 1,000.00   /overnight)
9th candidate
: published_hourly_price(Span("250.00", sentence=219837, chars=[31,36], words=[7,7]))
text for the 10th candidate:
Phrase (Doc: 8aa5b460-43f8-472c-81ed-efc6dd3485cf, Index: 44, Text: $ 80.00   /half hour,       $ 120.00   /hour)
10th candidate
: published_hourly_price(Span("120.00", sentence=230844, chars=[30,35], words=[7,7]))
#########################
text for the 12th candidate:
Phrase (Doc: 061b72cd-0d89-44e6-9a6f-7324d937286e, Index: 44, Text: $ 150.00   /half hour,       $ 200.00   /hour     ,   $ 300.00   /2 hours    ,   $ 600.00   /overnight)
12th candidate
: published_hourly_price(Span("200.00", sentence=196737, chars=[31,36], words=[7,7]))
########################################
text for the 100th candidate:
Phrase (Doc: aed7fee8-ede0-460c-bdf7-439ea797b707, 

235

In [31]:
right_ngrams_price_filter(cands200)

published_hourly_price(Span("450.00", sentence=226831, chars=[2,7], words=[1,1]))

In [45]:
'''create an .xlxs from price candidate
    '''

import xlsxwriter
all_cands = session.query(Published_Hourly_Price).all()

c_1 = all_cands[1]

x=c_1.get_parent()
y=str(x)

def doc_name_extractor(inputstr):
    return inputstr.get_parent().document.name
#     p = inputstr.get_parent()
#     y = str(p)
#     return y.split("(")[1].split(",")[0].split(": ")[1]

def price(data):
    return data[0].get_span()

# doc_name_extraction(y)
dict_cands ={}

for c in all_cands:
    file_name = doc_name_extractor(c)
    curr_price = price(c)
    #if curr_loc =="United States":
        #curr_loc = "USA"
    if file_name in dict_cands.keys():
        dict_cands[file_name].append(curr_price.lower())
        dict_cands[file_name] =list(set( dict_cands[file_name]))
    else:
        
        dict_cands[file_name] = [curr_price.lower()]

workbook = xlsxwriter.Workbook('texas_hourly_price_ver3.xlsx')
worksheet = workbook.add_worksheet()

d = dict_cands
row = 0
col = 0
worksheet.write(0, 0, "file name")
worksheet.write(0, 1, "price_pred")
for key in d.keys():
    row += 1
    worksheet.write(row, col, key)
    
    worksheet.write(row, col + 1, ','.join(map(str,list(set(d[key])))))
workbook.close()
        